In [444]:
__author__ = "Miguel Barreto Sanz"
__copyright__ = "Copyright 2018, Miguel Barreto Sanz"
__credits__ = ["Miguel Barreto Sanz"]
__license__ = "MIT"
__version__ = "0.0.1"
__maintainer__ = "Miguel Barreto Sanz"
__email__ = "miguelbarretosanz@gmail.com"
__status__ = "Development"

from collections import OrderedDict
from math import log, sqrt
import numpy as np
import pandas as pd
#six is a package that helps in writing code that is compatible with both Python 2 and Python 3.
from six.moves import cStringIO as StringIO 
from bokeh.plotting import figure, show, output_file


#Import modules for interactive graphics
from bokeh.layouts import row, widgetbox
from bokeh.models import CustomJS, Slider
from bokeh.layouts import column
from bokeh.models import HoverTool

#Import modules for conversions to radians.
import math
#Import modules for time management and time zones
import time, pytz, datetime


output_file("Visualization_sunburst.html", title="Visualization_sunburst.py example")

#GETING DATA

#Read data from Life Cycle format
LC_data = pd.read_csv('../data/Life Cycle/example/LC_export 3.csv')

#END -- GETING DATA


#PLOTING GRAPH
#size of the whole graphic
width = 700
height = 700

p = figure(plot_width=width, plot_height=height, title="",
    x_axis_type=None, y_axis_type=None,
    x_range=(-420, 420), y_range=(-420, 420),
    min_border=0, outline_line_color="white",
    background_fill_color="#ffffff",
    tools=[hover,])

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#First ring (fr) parameters
fr_inner_radius = 140
fr_outer_radius = 200

#Second ring (sr) parameters
sr_inner_radius = fr_outer_radius+2
sr_outer_radius = fr_outer_radius+52

#third ring (tr) parameters
tr_inner_radius = fr_outer_radius+52+2, 
tr_outer_radius = fr_outer_radius+52+2+42

def plot_ring(ring_number, start_time, duration, label, color, h_color):
    "Create costumized annular_edges. start_time in HH:MM:SS"
    if ring_number == 1:
        iner_radio = fr_inner_radius
        outer_radio = fr_outer_radius
    elif ring_number == 2:
        iner_radio = sr_inner_radius
        outer_radio = sr_outer_radius
    elif ring_number == 3:
        iner_radio = tr_inner_radius
        outer_radio = tr_outer_radius   
    else:
        print("ring_number value must be 1,2, or 3")  
    
    
    #Convert HH:MM:SS format in radians 
    ts = time.strptime(start_time, "%H:%M:%S") 
    hour = (ts[3] + (ts[4]/60) + (ts[5]/3600))
    hour_rad = math.radians(hour * 15.0)
    #add "pi/2" to transform radians to a 24 hours clock form.
    hour_in_radians_to_plot = -hour_rad + np.pi/2
    
    #Convert seconds in radians
    sec_rad = time.gmtime(duration)
    hour_duration = (sec_rad[3] + (sec_rad[4]/60))
    hour_rad_duration = math.radians(hour_duration * 15.0)
    duration_in_radians_to_plot = (hour_in_radians_to_plot + hour_rad_duration) 
    
    #The annular wedge is plotted in the direccion "anticlock". I tried to use the diection "clock" (which seems more 
    #logic for this plot) but I have some problems in the hit-testing. So I confirmed that it was a issue reported
    # with direction "clock : https://github.com/bokeh/bokeh/issues/2080
     
    p.annular_wedge(
        0, 0, iner_radio, outer_radio, hour_in_radians_to_plot - hour_rad_duration, duration_in_radians_to_plot - hour_rad_duration, 
        color=color, hover_color=h_color , name="anular_wedges"
        
    ) 
    
    return 


data1 = LC_data['START DATE(UTC)'][1]
#Parse string into a naive datetime object.
LCday,LCtime = data1.split(" ",1)

data2 = LC_data['START DATE(UTC)'][3]
#Parse string into a naive datetime object.
LCday2,LCtime2 = data2.split(" ",1)

data3 = LC_data['START DATE(UTC)'][4]
#Parse string into a naive datetime object.
LCday3,LCtime3 = data3.split(" ",1)


#First ring
plot_ring(1,LCtime,LC_data[' DURATION'][1],"label","cyan","magenta")
#Second ring
plot_ring(1,LCtime2,LC_data[' DURATION'][3],"label","red","pink")
#Third ring
plot_ring(1,LCtime3,LC_data[' DURATION'][4],"label","green","grey")

#PLOT CLOCK
clock_hours_to_plot = list(range(0,24))
angles = 2*np.pi/24*pd.Series(list(range(0,24)))
# radial axes
p.annular_wedge(0, 0, fr_inner_radius, tr_outer_radius, angles, angles, color="lightgrey")

# hours labels
minr = sqrt(log(.001 * 1E4))
maxr = sqrt(log(1000 * 1E4))
a = ((tr_outer_radius + 10) - fr_inner_radius) / (minr - maxr)
b = fr_inner_radius - a * maxr
radii = a * np.sqrt(np.log(labels * 1E4)) + b
xr = radii[0]*np.cos(np.array(angles))
yr = radii[0]*np.sin(np.array(angles))
label_angle=np.array(angles)
label_angle[label_angle < -np.pi/2] += np.pi # easier to read labels on the left side
labels_24h_clock = list(range(6,-1,-1)) + list(range(23,6,-1))

p.text(xr, yr, pd.Series(labels_24h_clock), angle=label_angle,
       text_font_size="9pt", text_align="center", text_baseline="middle", text_color="lightgrey")


#END -- PLOTING GRAPH



#CODE FOR THE INTEACTIVE PLOT

#slider to change the plot size
callback = CustomJS(args=dict(xr=p.x_range),code="""
    var a = -420;

    //the model that triggered the callback is cb_obj:
    var b = cb_obj.value;

    //models passed as args are automagically available
    xr.start = a;
    xr.end = b;
    
""")

# execute a callback whenever p.x_range.start changes
p.x_range.js_on_change('start', callback)
slider_x_range = Slider(start=10, end=1000, value=1, step=10, title="x_range", callback=callback)


#List of the plots which will use hover
hover = HoverTool(names=["anular_wedges"])


#END -- CODE FOR THE INTEACTIVE PLOT

#show plot and slider
layout = column(slider_x_range, p)
show(layout)


In [95]:
LC_data = pd.read_csv('../data/Life Cycle/example/LC_export 3.csv')
list(LC_data)

['START DATE(UTC)',
 ' END DATE(UTC)',
 ' START TIME(LOCAL)',
 ' END TIME(LOCAL)',
 ' DURATION',
 ' NAME',
 ' LOCATION']

In [106]:
LC_data['START DATE(UTC)'][2]

'2017-01-23 09:46:18'

In [ ]:
LC_data[' END DATE(UTC)']

In [196]:
LC_data

,START DATE(UTC),END DATE(UTC),START TIME(LOCAL),END TIME(LOCAL),DURATION,NAME,LOCATION
0,2017-01-22 10:11:30,2017-01-23 09:46:11,2017-01-22 11:11:30 CET,2017-01-23 10:46:11 CET,84881,Home,
1,2017-01-22 23:39:54,2017-01-23 07:00:02,2017-01-23 00:39:54 CET,2017-01-23 08:00:02 CET,26408,Sleep,
2,2017-01-23 09:46:18,2017-01-23 09:52:19,2017-01-23 10:46:18 CET,2017-01-23 10:52:19 CET,361,Waiting Bus,
3,2017-01-23 09:52:19,2017-01-23 10:00:53,2017-01-23 10:52:19 CET,2017-01-23 11:00:53 CET,514,Transport,
4,2017-01-23 10:00:53,2017-01-23 10:29:04,2017-01-23 11:00:53 CET,2017-01-23 11:29:04 CET,1691,ORP,ORP
5,2017-01-23 10:29:04,2017-01-23 10:33:40,2017-01-23 11:29:04 CET,2017-01-23 11:33:40 CET,276,Walk,
6,2017-01-23 10:40:49,2017-01-23 10:45:01,2017-01-23 11:40:49 CET,2017-01-23 11:45:01 CET,252,Transport,
7,2017-01-23 10:45:47,2017-01-24 08:55:12,2017-01-23 11:45:47 CET,2017-01-24 09:55:12 CET,79765,Home,
8,2017-01-23 23:52:41,2017-01-24 07:30:03,2017-01-24 00:52:41 CET,2017-01-24 08:30:03 CET,27442,Sleep,
9,2017-01-24 08:55:12,2017-01-24 09:04:35,2017-01-24 09:55:12 CET,2017-01-24 10:04:35 CET,563,Walk,


In [302]:
time.strptime("09:46:18", "%H:%M:%S")

time.struct_time(tm_year=1900, tm_mon=1, tm_mday=1, tm_hour=9, tm_min=46, tm_sec=18, tm_wday=0, tm_yday=1, tm_isdst=-1)

In [327]:
import time, pytz, datetime
data1 = LC_data['START DATE(UTC)'][2]
naive = datetime.datetime.strptime (data1, "%Y-%m-%d %H:%M:%S")
naive

datetime.datetime(2017, 1, 23, 9, 46, 18)

In [432]:
data1 = LC_data['START DATE(UTC)'][4]
data1
aa, bb = data1.split(" ",1)
aa
bb

'10:00:53'

In [434]:
print(LCtime2,LCtime3,LCtime)

09:46:18 09:46:18 09:46:18
